# Batch models analysis

## Purpose
- Overview of batch output
    - Count of model in each setting grid
    - Word accuracy at last epoch
    - Nonword accuracy at last epoch
- Use overview to control two plots
    - Development plot
    - W vs. NW plot
- Can "zoom-in" to specific run if needed

In [ ]:
%load_ext lab_black
import os, json
import pandas as pd
import altair as alt
import numpy as np
from meta import check_cfgs_params
from evaluate import make_df_wnw

alt.data_transformers.disable_max_rows()
alt.data_transformers.enable("default")

## Read files

In [ ]:
batch_name = "O2P_replication_set12"
batch_output_dir = "batch_eval/{}/".format(batch_name)
cfgs = pd.read_csv(batch_output_dir + "cfgs.csv", index_col=0)
df = pd.read_csv(batch_output_dir + "bcdf.csv", index_col=0)
n_rng = len(df.rng_seed.unique())
check_cfgs_params(cfgs)

### Explicitly provide varying h-params after reviewing unique params

In [ ]:
# variates = ["p_noise"]
variates = ["hidden_units", "learning_rate", "p_noise"]

# View h-param grid

In [ ]:
plot_n = (
    alt.Chart(cfgs)
    .mark_rect()
    .encode(
        x="p_noise:O",
        y="hidden_units:O",
        column="learning_rate:O",
        color="count(code_name)",
        tooltip=["count(code_name)"],
    )
    .properties(title="Model counts")
)

plot_n

### Check standard deviation for anomaly

In [ ]:
def plot_std(df, variates, conditions, dv):
    """
    Plot standard deviation of VARIABLE at given CONDITIONS at LAST TIME STEP
    df: pandas dataframe containing batch condition data file (bcdf)
    variates: a list of varying hyperparameters
    conditions: filter by conditions at df.cond
    dv: dependent variable to plot on heatmap (e.g., acc, sse)
    """
    # Select useful data
    sel_df = df.loc[
        (df.timestep == df.timestep.max()) & df.cond.isin(conditions),
        variates + ["cond", "epoch", "rng_seed", dv],
    ]

    # Collapse condition
    mean_df = sel_df.groupby(variates + ["epoch", "rng_seed"]).mean().reset_index()
    # Calculate standard deveiation in each cell
    plot_df = mean_df.groupby(variates + ["epoch"]).std().reset_index()

    # Plot heatmap
    return (
        alt.Chart(plot_df)
        .mark_rect()
        .encode(
            x="epoch:O",
            y="hidden_units:O",
            row="learning_rate:O",
            column="p_noise:O",
            color=alt.Color(dv, scale=alt.Scale(domain=[0, 0.2])),
            tooltip=variates + [dv],
        )
    )

# Examine rng_seed variation
### Plot all standard deviation if more than one rng_seed is found

In [ ]:
if n_rng > 1:

    # Strain SD
    sd_strain = plot_std(df, variates, ["INC_HF"], "acc")
    sd_strain.save(batch_output_dir + "stdev_strain_INCHF.html")

    # Grain SD
    sd_grain = plot_std(df, variates, ["unambiguous"], "acc")
    sd_grain.save(batch_output_dir + "stdev_grain_unambiguous.html")

    # Taraban SD
    taraban_w = [
        "High-frequency exception",
        "High-frequency regular-inconsistent",
        "Low-frequency exception",
        "Low-frequency regular-inconsistent",
        "Regular control for High-frequency exception",
        "Regular control for High-frequency regular-inconsistent",
        "Regular control for Low-frequency exception",
        "Regular control for Low-frequency regular-inconsistent",
    ]

    sd_taraban = plot_std(df, variates, taraban_w, "acc")
    sd_taraban.save(batch_output_dir + "stdev_taraban_all.html")

    # Glushko SD
    glushko_nw = ["Exception", "Regular"]
    sd_glushko = plot_std(df, variates, glushko_nw, "acc")
    sd_glushko.save(batch_output_dir + "stdev_glushko.html")

# All runs at last time step

### no rng_seed aggregation

In [ ]:
def plot_fig2(df):

    tmp = df.loc[df.timestep == df.timestep.max()]
    pdf = make_df_wnw(tmp, ["INC_HF"], ["unambiguous"])

    base = (
        alt.Chart(pdf)
        .mark_point()
        .encode(
            y=alt.Y("nonword_acc:Q", scale=alt.Scale(domain=(0, 1))),
            x=alt.X("word_acc:Q", scale=alt.Scale(domain=(0, 1))),
            color=alt.Color(
                "epoch", scale=alt.Scale(scheme="redyellowgreen", domain=(0, 100))
            ),
            opacity=alt.value(0.3),
            tooltip=["code_name", "epoch", "word_acc", "nonword_acc"],
        )
    )

    diagonal = (
        alt.Chart(pd.DataFrame({"x": [0, 1], "y": [0, 1]}))
        .mark_line(color="black")
        .encode(x="x", y="y")
    )
    return diagonal + base


plot_fig2(df).properties(title="All runs overlay (no aggregation)").save(
    batch_output_dir + "fig2.html"
)

### Plot figure 2 facet by regularization constant

### rng_seed aggregate

In [ ]:
if n_rng > 1:
    df["code_name"] = df.batch_unique_setting_string
    plot_fig2(df).properties(
        title="All runs overlay (within setting cell aggregation)"
    ).save(batch_output_dir + "fig2_agg.html")

### Create df for plotting

In [ ]:
if n_rng > 1:
    cfgs["code_name"] = cfgs.batch_unique_setting_string
    df["code_name"] = df.batch_unique_setting_string


pivotvars = variates + ["code_name", "epoch", "timestep", "unit_time", "exp", "cond"]
selvars = pivotvars + ["acc", "sse"]

df_cell_mean = df[selvars].pivot_table(index=pivotvars).reset_index()

# Select by condition and last time steps
df_sel = df_cell_mean.loc[
    (df_cell_mean.timestep == df_cell_mean.timestep.max())
    & df_cell_mean.cond.isin(["INC_HF", "unambiguous"])
]

# Make file
df_wnw = make_df_wnw(df_sel, ["INC_HF"], ["unambiguous"])
df_wnw["word_advantage"] = df_wnw.word_acc - df_wnw.nonword_acc
df_wnw = df_wnw.merge(cfgs)

selvars_wnw = variates + [
    "code_name",
    "epoch",
    "word_acc",
    "nonword_acc",
    "word_advantage",
]
df_wnw = df_wnw[selvars_wnw]

In [ ]:
heatmap_last = (
    alt.Chart(df_wnw.loc[df_wnw.epoch == df_wnw.epoch.max()])
    .mark_rect()
    .encode(
        x="p_noise:O",
        y="hidden_units:O",
        row="learning_rate:O",
        color=alt.Color(
            "word_acc:Q", scale=alt.Scale(scheme="redyellowgreen", domain=(0, 1))
        ),
        tooltip=["word_acc", "nonword_acc", "word_advantage"] + variates,
    )
    .properties(title="Last epoch word accuracy")
)

### Heatmap over epoch

In [ ]:
def heatmap(df_wnw, prefix=""):

    hm_base = (
        alt.Chart(df_wnw)
        .mark_rect()
        .encode(
            x="p_noise:O",
            y="hidden_units:O",
            row="learning_rate:O",
            column="epoch:O",
            tooltip=["word_acc", "nonword_acc", "word_advantage"] + variates,
        )
    )

    # Word
    hm_word = hm_base.encode(
        color=alt.Color(
            "word_acc:Q", scale=alt.Scale(scheme="redyellowgreen", domain=(0, 1))
        )
    ).properties(title="Word accuracy")
    # Nonword
    hm_nonword = hm_base.encode(
        color=alt.Color(
            "nonword_acc:Q", scale=alt.Scale(scheme="redyellowgreen", domain=(0, 1))
        )
    ).properties(title="Nonword accuracy")

    # Word - Nonword
    hm_wordadvantage = hm_base.encode(
        color=alt.Color(
            "word_advantage:Q",
            scale=alt.Scale(scheme="redyellowgreen", domain=(-0.3, 0.3)),
        )
    ).properties(title="Word advantage")

    hm_word.save(batch_output_dir + prefix + "heatmap_word.html")
    hm_nonword.save(batch_output_dir + prefix + "heatmap_nonword.html")
    hm_wordadvantage.save(batch_output_dir + prefix + "heatmap_wordadvantage.html")

In [ ]:
heatmap(df_wnw)

# Dashboard

In [ ]:
alt.data_transformers.disable_max_rows()


def main_dashboard(df):

    sel_run = alt.selection(type="multi", on="click", fields=["code_name"])

    # df for overview
    df_ov = df[df.epoch == df.epoch.max()]

    # Shared master over-view
    overview = (
        alt.Chart(df_ov)
        .mark_rect()
        .encode(
            x="p_noise:O",
            y="hidden_units:O",
            row="learning_rate:O",
            color=alt.Color(
                "word_acc", scale=alt.Scale(scheme="redyellowgreen", domain=(0, 1))
            ),
            opacity=alt.condition(sel_run, alt.value(1), alt.value(0.1)),
            tooltip=[
                "code_name",
                "p_noise",
                "hidden_units",
                "learning_rate",
                "word_acc",
                "nonword_acc",
            ],
        )
        .add_selection(sel_run)
        .properties(title="Word accuracy at the end of training")
    )

    wnw_mdf = df.melt(
        id_vars=["code_name", "epoch"],
        value_vars=["word_acc", "nonword_acc"],
        var_name="wnw",
        value_name="acc",
    )

    plot_epoch = (
        alt.Chart(wnw_mdf)
        .mark_point(size=80)
        .encode(
            y=alt.Y("acc:Q", scale=alt.Scale(domain=(0, 1))),
            x="epoch:Q",
            color=alt.Color("code_name:N", legend=None),
            shape="wnw:N",
            opacity=alt.condition(sel_run, alt.value(1), alt.value(0)),
            tooltip=["code_name", "epoch", "acc"],
        )
        .add_selection(sel_run)
        .properties(title="Plot word and nonword accuracy by epoch")
    )

    wnw_line = (
        alt.Chart(df)
        .mark_line()
        .encode(
            y=alt.Y("nonword_acc:Q", scale=alt.Scale(domain=(0, 1))),
            x=alt.X("word_acc:Q", scale=alt.Scale(domain=(0, 1))),
            color=alt.Color("code_name:N", legend=None),
            opacity=alt.condition(sel_run, alt.value(1), alt.value(0)),
            tooltip=["code_name", "epoch", "word_acc", "nonword_acc"],
        )
    )

    wnw_point = wnw_line.mark_point().encode(
        color=alt.Color("epoch", scale=alt.Scale(scheme="redyellowgreen"))
    )

    diagonal = (
        alt.Chart(pd.DataFrame({"x": [0, 1], "y": [0, 1]}))
        .mark_line(color="black")
        .encode(x="x", y="y")
    )

    wnw = diagonal + wnw_line + wnw_point

    wnw_interactive = wnw.add_selection(sel_run).properties(
        title="Word vs. Nonword accuracy at final time step"
    )

    ### Mini heatmap ###

    mini_wadv = (
        alt.Chart(df)
        .mark_rect()
        .encode(
            x="epoch:O",
            color=alt.Color(
                "word_advantage:Q",
                scale=alt.Scale(scheme="redyellowgreen", domain=(-0.3, 0.3)),
            ),
            opacity=alt.condition(sel_run, alt.value(1), alt.value(0)),
            tooltip=["word_acc", "nonword_acc", "word_advantage"] + variates,
        )
        .properties(title="Word - Nonword")
    )

    return overview | (plot_epoch & mini_wadv) | wnw_interactive


main_dashboard(df_wnw).save(batch_output_dir + "dashboard.html")

### Hyper-parameter effect plots

In [ ]:
plot_pnoise = (
    alt.Chart()
    .mark_line()
    .encode(
        y=alt.Y("nonword_acc:Q", scale=alt.Scale(domain=(0, 1))),
        x=alt.X("word_acc:Q", scale=alt.Scale(domain=(0, 1))),
        color=alt.Color("p_noise:O", scale=alt.Scale(scheme="reds")),
        tooltip=variates + ["epoch", "word_acc", "nonword_acc"],
    )
)

diagonal = (
    alt.Chart(pd.DataFrame({"x": [0, 1], "y": [0, 1]}))
    .mark_line(color="black")
    .encode(x="x", y="y")
)

p = alt.layer(diagonal + plot_pnoise, data=df_wnw).facet(
    row="hidden_units:O", column="learning_rate:O"
)

p.save(batch_output_dir + "Effect_pnoise.html")

In [ ]:
plot_hidden = (
    alt.Chart()
    .mark_line()
    .encode(
        y=alt.Y("nonword_acc:Q", scale=alt.Scale(domain=(0, 1))),
        x=alt.X("word_acc:Q", scale=alt.Scale(domain=(0, 1))),
        color=alt.Color("hidden_units:O", scale=alt.Scale(scheme="blues")),
        tooltip=variates + ["epoch", "word_acc", "nonword_acc"],
    )
)

h = alt.layer(diagonal + plot_hidden, data=df_wnw).facet(
    row="p_noise:O", column="learning_rate:O"
)

h.save(batch_output_dir + "Effect_hidden.html")

In [ ]:
plot_lr = (
    alt.Chart()
    .mark_line()
    .encode(
        y=alt.Y("nonword_acc:Q", scale=alt.Scale(domain=(0, 1))),
        x=alt.X("word_acc:Q", scale=alt.Scale(domain=(0, 1))),
        color=alt.Color(
            "learning_rate", type="ordinal", scale=alt.Scale(scheme="greens")
        ),
        tooltip=variates + ["epoch", "word_acc", "nonword_acc"],
    )
)

lr = alt.layer(diagonal + plot_lr, data=df_wnw).facet(
    row="hidden_units:O", column="p_noise:O"
)

lr.save(batch_output_dir + "Effect_lr.html")

SSE vs. P-noise

In [ ]:
sdf = df.loc[
    (
        (df.timestep == df.timestep.max())
        & (df.cond.isin(["CON_HF", "CON_LF", "INC_HF", "INC_LF"]))
    ),
    ["code_name", "epoch", "p_noise", "acc", "sse"],
]

# Collapse condition
sdf = sdf.groupby(["code_name", "epoch"]).mean().reset_index()

In [ ]:
alt.data_transformers.disable_max_rows()
c = (
    alt.Chart(sdf)
    .mark_point()
    .encode(y="acc", x=alt.X("sse", scale=alt.Scale(domain=(0, 8))), color="p_noise:N",)
    .interactive()
)
c